In [14]:
# ⬇️ Cell: hot-reloading switch/ICMP controls with dry-run support
import time, importlib, ipywidgets as widgets
from IPython.display import display

import switch_config   # keep a handle so we can reload it later

# -- quick toggles ------------------------------------------------------------
DISABLE_ICMP = False   # True → don't touch ICMP controller
DISABLE_SW   = False   # True → don't touch hardware; just print config voltages

from switch_config import HostInformation
if not DISABLE_SW: 
    from switch_ctrl import SwitchController
if not DISABLE_ICMP:
    from icmp_ctrl import ICMPController, get_public_ip
    
    
# -- controller instances -----------------------------------------------------
switch_controller = None
icmp_controller   = None

# -- helper: (re)load configs --------------------------------------------------
def reload_configs():
    """
    Re-import switch_config so edits on disk take effect immediately.
    Also push the refreshed configs into any live controller objects.
    """
    global switch_config, SW_CONFIGS, HOSTS
    switch_config = importlib.reload(switch_config)

    # pull the refreshed constants
    SW_CONFIGS      = switch_config.SW_CONFIGS
    HOSTS           = switch_config.HOSTS

    # update existing controllers if they expose the same attr
    if not DISABLE_SW and switch_controller is not None:
        if hasattr(switch_controller, "sw_configs"):
            switch_controller.sw_configs = SW_CONFIGS
    if not DISABLE_ICMP and icmp_controller is not None:
        if hasattr(icmp_controller, "sw_configs"):
            icmp_controller.sw_configs = SW_CONFIGS

# first load
reload_configs()

# -- import the rest *after* configs exist ------------------------------------
from switch_config import HostInformation          # HostInformation is stable

if not DISABLE_SW:
    from switch_ctrl import SwitchController
if not DISABLE_ICMP:
    from icmp_ctrl import ICMPController, get_public_ip

PRE_GUARD_MS, POST_GUARD_MS = 10, 5000


if not DISABLE_SW:
    switch_controller = SwitchController("/dev/ttyUSB1", SW_CONFIGS)
    switch_controller.zero(verify=False)

if not DISABLE_ICMP:
    icmp_controller = ICMPController(get_public_ip(), hosts=HOSTS, sw_configs=SW_CONFIGS)
    
reload_configs()

# -- UI widgets ----------------------------------------------------------------
out_log  = widgets.Output(layout={"border": "1px solid #ccc", "max_height": "500px", "overflow": "auto"})
btn_cfg1 = widgets.Button(description="Config 1", button_style="success")
btn_cfg2 = widgets.Button(description="Config 2", button_style="info")
btn_zero = widgets.Button(description="Zero",    button_style="warning")

# -- internal helpers ----------------------------------------------------------
def _pretty_dump(cfg):
    """Human-friendly dump of relay/voltage pairs for dry-run mode."""
    try:
        for ch, v in cfg.ring_configs:
            print(f"  Ring {ch:<2}: {v} V")
    except Exception:
        print(cfg)

def apply_config(cfg_idx: int):
    reload_configs()                      # ← hot-reload on every click
    cfg = SW_CONFIGS[cfg_idx]

    out_log.clear_output(wait=True)
    with out_log:
        print(f"▶ Switching to configuration {cfg_idx + 1} …")

    if not DISABLE_ICMP:
        icmp_controller.reconfig_start(cfg_idx)
    time.sleep(PRE_GUARD_MS/1000)

    if not DISABLE_SW:
        switch_controller.zero(verify=False)
        switch_controller.set_configuration(cfg_idx)
    else:
        with out_log:
            print("— Hardware disabled; dry-running:")
            _pretty_dump(cfg)

    time.sleep(POST_GUARD_MS/1000)

    if not DISABLE_ICMP:
        icmp_controller.reconfig_finish(cfg_idx)
    else:
        print("ICMP disabled")

    with out_log:
        print(f"✓ Now at configuration {cfg_idx + 1}.\n")

def zero_switch(_btn=None):
    reload_configs()                      # keep parity with apply_config
    out_log.clear_output(wait=True)

    if not DISABLE_SW:
        with out_log:
            print("▶ Zeroing switch …")
        switch_controller.zero(verify=False)
        with out_log:
            print("✓ Switch zeroed.")
    else:
        with out_log:
            print("— Hardware disabled; nothing to zero.")

# -- wire callbacks & display --------------------------------------------------
btn_cfg1.on_click(lambda _ : apply_config(0))
btn_cfg2.on_click(lambda _ : apply_config(1))
btn_zero.on_click(zero_switch)

display(widgets.HBox([btn_cfg1, btn_cfg2, btn_zero]))
display(out_log)


INFO:switch_ctrl:Qontroller 'Q8iv-037E' initialised with firmware v2.4.1 and 96 channels
INFO:switch_ctrl:
Zeroing Complete


Output(layout=Layout(border_bottom='1px solid #ccc', border_left='1px solid #ccc', border_right='1px solid #cc…